## Código de preparación de datos...

Este pequeño código prepara los datos para ser manipulados y ser usados por otro código de propagación de muones en roca.. La idea es que tenemos dos archivos de datos. El primero es la salida de muones del volcán machin en el punto P1 de observación, el otro archivo de datos es la distancia recorrida por los muones en roca, calculado a partir de un código en matlab externo. La idea es manipular el primero de los archivos para obtener la energía total a partir de la consideración de los momentos en las direcciones $x$, $y$, y $z$. Lo segundo es teniendo la energía vamos a organizar los datos de tal manera de calcular el flujo, es decir, sumar todos los muones que llegan en la misma dirección para tener el número de muones que entran por esa determinada dirección, con esto tendremos una data de flujo por dirección.
Lo siguiente es combinar el archivo de datos de distancias recorridas y le ponga a cada partícula entrante con energía determinada E, su correspondiente distancia recorrida...

In [1]:
# Cargando todas las librerías que necesitamos
import numpy as np
import math
import pandas as pd 
import matplotlib.pyplot as plt 


theta1, theta2= 18, 35
############################
df1=pd.read_csv('./salida_d_2.shw',sep =" ", header=None)
df1=df1.drop(index=range(6))  
df2 = df1[df1.columns[1:4]] 
px = df2[df2.columns[:1]].to_numpy()
py = df2[df2.columns[1:2]].to_numpy()
pz = df2[df2.columns[2:3]].to_numpy()
pt=np.sqrt(px**2+py**2+pz**2) 

Llamamos al primer archivo de datos originales que vienen de Corsika...

Acá ahora tomamos los momentos y calculamos la energía total de cada muon que llega al volcán... En este caso 
\begin{equation}
\theta=\arccos\frac{p_z}{p}=\arccos\frac{p_z}{\sqrt{p_x^2+p_y^2+p_z^2}}, \ \ \phi=\arctan\frac{p_y}{p_x}
\end{equation}

In [2]:
pt=np.sqrt(px**2+py**2+pz**2)
m_rest=0.1057 #unit: GeV/c**2
Et=np.sqrt(pt**2+m_rest**2)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
thetat=np.degrees(np.arccos(pz/pt))
at2=np.arctan2(py,px)
phit=np.degrees((2*np.pi+at2)*(at2<0)+(at2)*(at2>0))

#Ahora tenemos un archivo de ángulos de incidencia vs. energía con la que llegan. 
#Guardamos los datos nuevos con el formato requerido... Cargamos la nueva dataframe y ordenamos por ángulos.

data=np.c_[phit,thetat,Et] #theta ya esta en grados
data=data[data[:,1].argsort()] # Ordena de acuerdo a angulo theta  

# Obtenemos solo los que están en el rango 117<phi<147 y 66<theta<84
data=data[(data[:,0]>=89) & (data[:,0]<=91) & (data[:,1]>=theta1) & (data[:,1]<=theta2)] #--- acá le pongo la condición 
data=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',data,delimiter=' ',fmt='%i %i %2f')

df3=pd.read_csv('totalrealCerroUNI_tpE_ordenados.out',sep=' ',names=['phi','theta','E'])
dfeventos = df3[df3["phi"]==90]
datac=np.savetxt('totalrealCerroUNI_tpE_ordenados.out',dfeventos,delimiter=' ',fmt='%i %i %2f')


In [3]:
#arrf = thetat[(thetat >= theta1) & (thetat<=theta2)] #thetat es el valor de ángulo azimutal
#print(type(arrf[0]))
#plt.hist(arrf, bins=50)
dfeventos #eventos en el shower ordenados por theta 


,phi,theta,E
0,90,18,0.169939
1,90,18,0.105702
3,90,18,0.105745
6,90,18,0.433390
7,90,18,0.105703
...,...,...,...
2456,90,34,0.106306
2457,90,34,0.105703
2459,90,34,0.105700
2461,90,34,0.105700


In [4]:
#conteo de los valores a cielo abierto
df6=dfeventos.groupby(['phi','theta'])['E'].count().reset_index(name='count')
df6.to_csv('totalrealCerroUNI_tpcount_ordenados_cielo_abierto.out',sep=' ',index=False)
df6 #conteo de eventos en el shower dependiendo de theta a cielo abierto 

,phi,theta,count
0,90,18,74
1,90,19,72
2,90,20,66
3,90,21,77
4,90,22,73
5,90,23,64
6,90,24,81
7,90,25,86
8,90,26,65
9,90,27,70


In [5]:
df_distancia=pd.read_csv('datos_fila_210_angmax_48_apertura_30_pasos_19.txt',sep=' ',names=['fila','angulo','distancia'],skiprows=1)
df_distancia['Elevacion'] = 90-df_distancia['angulo']
df_distancia

,fila,angulo,distancia,Elevacion
0,210,72,29.489506,18
1,210,71,27.558188,19
2,210,70,27.327721,20
3,210,69,25.132888,21
4,210,68,24.898500,22
5,210,67,22.145449,23
6,210,66,21.718271,24
7,210,65,21.494758,25
8,210,64,19.214091,26
9,210,63,19.214091,27


In [88]:
#ecuacion  


def funciondEdX(E): 
    me = 0.0005285 #GeV/c2
    mu = 0.1057 #GeV/c2
    
    
    Emax = (E**2)/(E+(mu**2)/(2*me))
    dEdx = 1.84 + 0.076* np.log(Emax/(mu))

    return dEdx
        
        
def perdida_energia_total(E, d): #energia y distancia por recorrer
    
    
    temp = E
    ar_x = np.linspace(0, d, 100)
    
    for i in range(len(ar_x)): 
        
        temp = temp - funciondEdX(temp)
        print(temp)
        if temp < 0 : 
            control = False
            temp = 0
            break 
    
    energia_final = temp 
    control = True
    
    return (energia_final, control)
    
    
        

In [7]:
df_distancia

,fila,angulo,distancia,Elevacion
0,210,72,29.489506,18
1,210,71,27.558188,19
2,210,70,27.327721,20
3,210,69,25.132888,21
4,210,68,24.898500,22
5,210,67,22.145449,23
6,210,66,21.718271,24
7,210,65,21.494758,25
8,210,64,19.214091,26
9,210,63,19.214091,27


In [8]:

df2 = df_distancia[df_distancia.columns[2:4]] 
vec_distancia = df_distancia[df_distancia.columns[2:4]].to_numpy() 
vec_energia = dfeventos[dfeventos.columns[2:3]].to_numpy()

In [ ]:
for i  in range(len(df_distancia)): 
    print(vec_energia[i][0]) 

In [ ]:
val = perdida_energia_total(vec_energia[4][0], vec_distancia[0][0])
print(val[1])

In [ ]:
print(vec_distancia[0][0])
for i in np.arange(0,vec_distancia[0][0], 0.1): 
    print(i)

In [ ]:
for i in range(0,int(vec_distancia[0][0]), 1): 
    print(i)

In [9]:
veco=[]
#df_distancia.iloc[i][3]: df_distancia[eventos][3era columna]
#df_distancia

for j in range(len(dfeventos)):  
    val = True
    for i in range(0,int(vec_distancia[0][0]), 1): #i debería ser los eventos
        if dfeventos.iloc[j][1] == df_distancia.iloc[i][3]: 
            m = df_distancia.iloc[i][3]
            #atenuacion_form(dfeventos.iloc[j][2],df_distancia.iloc[i][3])
            #atenuacion_tabla(dfeventos.iloc[j][2],df_distancia.iloc[i][3])
            val = perdida_energia_total(vec_energia[4][0],vec_distancia[0][0])[1]
            print(val)
            if val == True: 
                veco.append([dfeventos.iloc[j][1], val]) 

True


IndexError: single positional indexer is out-of-bounds

In [ ]:
vecc = np.array(veco)

df_atenuado = pd.DataFrame(vecc)
df_atenuado.columns = ['Elevacion', 'Condicional']

print(df_atenuado) 

In [ ]:
df7=df_atenuado.groupby(['Elevacion'])['Condicional'].count().reset_index(name='count')
df7.to_csv('totalrealCerroUNI_tpcount_ordenados_atenuado.out',sep=' ',index=False)
df7

In [20]:
print(len(df_distancia))
df_distancia
print(vec_distancia[0][1])

17
18.0


In [32]:
diccionario = {}  # Inicializar el diccionario vacío

for i in range(0, len(df_distancia)):  # Bucle for para generar claves del 1 al 5
    clave = vec_distancia[i][1]
    valor = vec_distancia[i][0]
    diccionario[clave] = valor  # Agregar clave-valor al diccionario

print(type(diccionario[18]))
# Salida:
# {1: 1, 2: 4, 3: 9, 4: 16, 5: 25}




print(diccionario)


<class 'numpy.float64'>
{18.0: 29.489505992640836, 19.0: 27.558187641606622, 20.0: 27.32772094869969, 21.0: 25.13288770694685, 22.0: 24.89849987449043, 23.0: 22.145448742348844, 24.0: 21.718271468051963, 25.0: 21.494757761137954, 26.0: 19.21409064618984, 27.0: 19.21409064618984, 28.0: 16.54981736334272, 29.0: 13.927598674574158, 30.0: 13.604331833647691, 31.0: 11.08434672184157, 32.0: 8.798142676724455, 33.0: 6.760462115122006, 34.0: 3.748500533546716}


In [93]:
def acceder_diccionario(diccionario, valor_clave):
    valor_distancia = 0 
    for clave, valor in diccionario.items(): 
        if clave == valor_clave: 
            valor_distancia = diccionario[valor_clave]
    return valor_distancia


# Acceder al diccionario de forma recursiva
print(acceder_diccionario(diccionario, 18))
  

29.489505992640836


In [48]:
print(dfeventos.iloc[0][1])

,phi,theta,E
0,90,18,0.169939
1,90,18,0.105702
3,90,18,0.105745
6,90,18,0.433390
7,90,18,0.105703
...,...,...,...
2456,90,34,0.106306
2457,90,34,0.105703
2459,90,34,0.105700
2461,90,34,0.105700


In [141]:

def funciondEdX(E): 
    me = 0.0005285 #GeV/c2
    mu = 0.1057 #GeV/c2
    
    if E > 0 :    

        Emax = (E**2)/(E+(mu**2)/(2*me))
        dEdx = 1.84 + 0.076* np.log(Emax/(mu))
    else: 
        dEdx = 0

    return dEdx
        
        
def perdida_energia_total(E, d): #energia y distancia por recorrer
    temp = E 
    
    if E > 0: 
    
        ar_x = np.linspace(0, d, 10)
    
        for i in range(len(ar_x)): 

            temp = temp - funciondEdX(temp)
            
            #if temp < 0 : 
             #   control = False
              #  temp = 0
               # break 
    
    energia_final = temp 
    control = True
    return (energia_final, control)
    
    

In [145]:
print(len(dfeventos))
for j in range(len(dfeventos)): 
    
    temp_angulos_shower = dfeventos.iloc[j][1] # todos los ángulos 
    temp_energia_shower = dfeventos.iloc[j][2] * 1000 # todas las energías 
    temp_distancia_x_angulo = acceder_diccionario(diccionario, temp_angulos_shower)
    #print(temp_distancia_x_angulo, temp_energia_shower, temp_angulos_shower) funciona
    #print(funciondEdX(temp_energia_shower)) funciona
    if perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[1] == False: 
        print("energia final por evento j:", j, "= ", perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[0], "energía inicial: ", temp_energia_shower, "Condicional: " , perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[1])


1264


In [96]:
print(funciondEdX(temp_energia_shower))

1.4892522716264578


In [82]:
veco=[]
#df_distancia.iloc[i][3]: df_distancia[eventos][3era columna]
#df_distancia
energia_final_x_angulo = 0 
condicional_x_angulo = True
for j in range(len(dfeventos)):  
#for j in range(0,5,1):
    temp_angulos_shower = dfeventos.iloc[j][1] # todos los ángulos 
    temp_energia_shower = dfeventos.iloc[j][2] # todos los ángulos 
    temp_distancia_x_angulo = acceder_diccionario(diccionario, temp_angulos_shower)
    condicional_x_angulo = perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[1]
    print(perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[0],perdida_energia_total(temp_energia_shower, temp_distancia_x_angulo)[1])
    #if condicional_x_angulo == True: 
        #print(temp_angulos_shower, temp_energia_shower, temp_distancia_x_angulo, condicional_x_angulo, "dista;" , vec_distancia[0][0])
    


0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True
0 True

In [61]:
dfeventos

,phi,theta,E
0,90,18,0.169939
1,90,18,0.105702
3,90,18,0.105745
6,90,18,0.433390
7,90,18,0.105703
...,...,...,...
2456,90,34,0.106306
2457,90,34,0.105703
2459,90,34,0.105700
2461,90,34,0.105700
